In [1]:
from tqdm import tqdm
import json
from util import get_scorecard, maybe_read_json_file
import pandas as pd

terrapin_db = pd.read_csv("./data/terrapin.csv")
terrapin_db = terrapin_db[terrapin_db["SrcRepo"].notna()]
terrapin_data = maybe_read_json_file("./terrapin-scores.json")
print("Initial # Terrapin repos:", len(terrapin_data))

n_failed = 0
n_success = 0
recent_reason = ""
p_bar = tqdm(terrapin_db.iterrows())
for _, row in p_bar:
    p_bar.set_description(f"{n_success} success, {n_failed} failed, reason: {recent_reason}")
    if row["SrcRepo"] not in terrapin_data:
        success, maybe_scorecard = get_scorecard(f"github.com/{row['SrcRepo']}")
        if success:
            terrapin_data[row["SrcRepo"]] = maybe_scorecard
            with open("./terrapin-scores.json", "w") as f:
                json.dump(terrapin_data, f)
            n_success += 1
        else:
            n_failed += 1
            recent_reason = maybe_scorecard[:80]



Initial # Terrapin repos: 1818


207 success, 98 failed, reason: 2021/12/10 11:11:38 repo unreachable: GET https://api.github.com/repos/Matthewsr: : 7938it [1:14:21,  3.93s/it]